# Capstone Project - NICE RESTAURANT
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

In this project we will try to find an optimal location for a restaurant. Specifically, this report will be targeted to stakeholders interested in opening an **Italian restaurant** in **Nice**, France.

Since there are lots of restaurants in Nice we will try to detect **locations that are not already crowded with restaurants**. We are also particularly interested in **areas with no Italian restaurants in vicinity**. We would also prefer locations **as close to city center as possible**, assuming that first two conditions are met.

We will use our data science powers to generate a few most promissing neighborhoods based on these criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decision are:
* number of existing restaurants in the neighborhood (any type of restaurant)
* number of and distance to Italian restaurants in the neighborhood, if any
* distance of neighborhood from city center

We decided to use regularly spaced grid of locations, centered around city center, to define our neighborhoods.

Following data sources will be needed to extract/generate the required information:
* centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using **OpenStreetMap reverse geocoding**
* number of restaurants and their type and location in every neighborhood will be obtained using **Foursquare API**
* coordinate of Nice center will be obtained using **GeoPy API geocoding** of well known Nice location (Nice Etoile Jean Medecin)

In [22]:
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

address = 'Nice'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
nice_center = (latitude, longitude)
print('The geograpical coordinates of Nice are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Nice are 43.7009358, 7.2683912.


Now let's create a grid of area candidates, equaly spaced, centered around city center and within ~3km from Nice Center. Our neighborhoods will be defined as circular areas with a radius of 150 meters, so our neighborhood centers will be 300 meters apart.

To accurately calculate distances we need to create our grid of locations in Cartesian 2D coordinate system which allows us to calculate distances in meters (not in latitude/longitude degrees). Then we'll project those coordinates back to latitude/longitude degrees to be shown on Folium map. So let's create functions to convert between WGS84 spherical coordinate system (latitude/longitude degrees) and UTM Cartesian coordinate system (X/Y coordinates in meters).

In [23]:
!pip install shapely
import shapely.geometry

!pip install pyproj
import pyproj

import math

def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transformation check')
print('-------------------------------')
print('Nice center longitude={}, latitude={}'.format(nice_center[1], nice_center[0]))
x, y = lonlat_to_xy(nice_center[1], nice_center[0])
print('Nice center UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('Nice center longitude={}, latitude={}'.format(lo, la))


Requirement not upgraded as not directly required: shapely in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: pyproj in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Coordinate transformation check
-------------------------------
Nice center longitude=7.2683912, latitude=43.7009358
Nice center UTM X=-123064.11761566252, Y=4867792.654074785
Nice center longitude=7.268391200000001, latitude=43.700935799999996


Let's create a **hexagonal grid of cells**: we offset every other row, and adjust vertical row spacing so that **every cell center is equally distant from all it's neighbors**.

In [24]:
import requests

def get_postalCode( latitude, longitude, verbose=False):
    try:
        url = 'https://nominatim.openstreetmap.org/reverse?format=jsonv2&lat={}&lon={}'.format(latitude, longitude)
#        print(url)
#        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&latlng={},{}'.format(api_key, latitude, longitude)
        response = requests.get(url).json()
#    print(response)
        if verbose:
            print('Google Maps API JSON result =>', response)
#        results = response['results']
#        address = results[0]['display_name']
#        print(response['address']['postcode'])
        
 
        return response['address']['postcode']
    except:
        return None

#url = 'https://nominatim.openstreetmap.org/reverse?format=jsonv2&lat={}&lon={}'.format(latitude, longitude)
#response = requests.get(url).json()
#address = response['address']['postcode']
   
#print(address)
print(get_postalCode(43.7009358, 7.2683912))



06000


In [28]:
nice_center_x, nice_center_y = lonlat_to_xy(nice_center[1], nice_center[0]) # City center in Cartesian coordinates

k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_min = nice_center_x - 2000
x_step = 200
y_min = nice_center_y - 2000 - (int(21/k)*k*200 - 4000)/2
y_step = 200 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 100 if i%2==0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(nice_center_x, nice_center_y, x, y)
        if (distance_from_center <= 2001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)
         #   print(get_postalCode(lat, lon))

 # print(get_postalCode(43.7009358, 7.2683912))
print(len(latitudes), 'candidate neighborhood centers generated.')


364 candidate neighborhood centers generated.


Let's visualize the data we have so far: city center location and candidate neighborhood centers:

In [25]:
!pip install folium

import folium

Requirement not upgraded as not directly required: folium in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: numpy in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: jinja2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: branca>=0.3.0 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->folium)
Requirement not upgraded as not directly required: idna<2.7,

In [29]:
map_nice = folium.Map(location=nice_center, zoom_start=13)
folium.Marker(nice_center, popup='Nice Center').add_to(map_nice)
for lat, lon in zip(latitudes, longitudes):
    #folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_nice) 
    folium.Circle([lat, lon], radius=100, color='blue', fill=False).add_to(map_nice)
    #folium.Marker([lat, lon]).add_to(map_nice)
map_nice


OK, we now have the coordinates of centers of neighborhoods/areas to be evaluated, equally spaced (distance from every point to its neighbors is exactly the same) and within ~3km from Nice Etoile Jean Medecin. 

Let's now use OpenStreetMap API to get approximate addresses of those locations.


In [30]:
import requests

def get_address( latitude, longitude, verbose=False):
    try:
        url = 'https://nominatim.openstreetmap.org/reverse?format=jsonv2&lat={}&lon={}'.format(latitude, longitude)
#        print(url)
#        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&latlng={},{}'.format(api_key, latitude, longitude)
        response = requests.get(url).json()
#    print(response)
        if verbose:
            print('Google Maps API JSON result =>', response)
#        results = response['results']
#        address = results[0]['display_name']
        address = response['display_name']
        return address
    except:
        return None
'''
latitude = nice_center[0]
longitude = nice_center[1]
url = 'https://nominatim.openstreetmap.org/reverse?format=jsonv2&lat={}&lon={}'.format(latitude, longitude)
#print(url)
#response = requests.get(url).json()
#print(response)
#print(response['name'])
print(response['display_name'])
#latitude
'''
addr = get_address(nice_center[0], nice_center[1])

print('Reverse geocoding check')
print('-----------------------')
print('Address of [{}, {}] is: {}'.format(nice_center[0], nice_center[1], addr))


Reverse geocoding check
-----------------------
Address of [43.7009358, 7.2683912] is: Centre commercial Nice-Étoile, Avenue Jean Médecin, Quartier Jean-Médecin, Nice, Alpes-Maritimes, Provence-Alpes-Côte d'Azur, France métropolitaine, 06000, France


In [31]:
print('Obtaining location addresses: ', end='')
addresses = []
for lat, lon in zip(latitudes, longitudes):
    address = get_address(lat, lon)
    if address is None:
        address = 'NO ADDRESS'
    address = address.replace(', France', '') # We don't need country part of address
    addresses.append(address)
    print(' .', end='')
print(' done.')

Obtaining location addresses:  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


In [32]:
addresses[150:170]

["Max Barel, Boulevard Lech Walesa, Le Port, Nice, Alpes-Maritimes, Provence-Alpes-Côte d'Azur métropolitaine, 06300",
 "Impasse Victor Lavagna, Le Port, Nice, Alpes-Maritimes, Provence-Alpes-Côte d'Azur métropolitaine, 06300",
 "Lycée professionnel Les Palmiers, Avenue Banco, Nice Est, Nice, Alpes-Maritimes, Provence-Alpes-Côte d'Azur métropolitaine, 06300",
 "Avenue Louis Janion, L'Archet, Grosso, Nice, Alpes-Maritimes, Provence-Alpes-Côte d'Azur métropolitaine, 06046",
 "74, Avenue des Baumettes, L'Archet, Grosso, Nice, Alpes-Maritimes, Provence-Alpes-Côte d'Azur métropolitaine, 06046",
 "Les Orchidées, Avenue des Fleurs, L'Archet, Gambetta, Nice, Alpes-Maritimes, Provence-Alpes-Côte d'Azur métropolitaine, 06046",
 "15, Rue Caffarelli, L'Archet, Gambetta, Nice, Alpes-Maritimes, Provence-Alpes-Côte d'Azur métropolitaine, 06046",
 "Orpi Actuel Immo, 39, Boulevard Gambetta, Gambetta, Nice, Alpes-Maritimes, Provence-Alpes-Côte d'Azur métropolitaine, 06000",
 "Le Victor Hugo, Rue Berlioz

Looking good. Let's now place all this into a Pandas dataframe.

In [34]:
import pandas as pd

df_locations = pd.DataFrame({'Address': addresses,
                             'Latitude': latitudes,
                             'Longitude': longitudes,
                             'X': xs,
                             'Y': ys,
                             'Distance from center': distances_from_center})

df_locations.head(10)

,Address,Distance from center,Latitude,Longitude,X,Y
0,"Gambetta, Nice, Alpes-Maritimes, Provence-Alpe...",1997.498436,43.683435,7.263212,-123664.117616,4.865887e+06
1,"Massena, Nice, Alpes-Maritimes, Provence-Alpes...",1946.792233,43.683602,7.265670,-123464.117616,4.865887e+06
2,"Massena, Nice, Alpes-Maritimes, Provence-Alpes...",1915.724406,43.683770,7.268129,-123264.117616,4.865887e+06
3,"Massena, Nice, Alpes-Maritimes, Provence-Alpes...",1905.255888,43.683937,7.270588,-123064.117616,4.865887e+06
4,"Massena, Nice, Alpes-Maritimes, Provence-Alpes...",1915.724406,43.684104,7.273047,-122864.117616,4.865887e+06
5,"Massena, Nice, Alpes-Maritimes, Provence-Alpes...",1946.792233,43.684272,7.275506,-122664.117616,4.865887e+06
6,"Massena, Nice, Alpes-Maritimes, Provence-Alpes...",1997.498436,43.684439,7.277965,-122464.117616,4.865887e+06
7,"Grosso, Nice, Alpes-Maritimes, Provence-Alpes-...",1951.922130,43.684729,7.259324,-123964.117616,4.866061e+06
8,"Gambetta, Nice, Alpes-Maritimes, Provence-Alpe...",1868.154169,43.684897,7.261782,-123764.117616,4.866061e+06
9,"Massena, Nice, Alpes-Maritimes, Provence-Alpes...",1802.775638,43.685064,7.264241,-123564.117616,4.866061e+06


In [36]:
df_locations.to_pickle('./locations.pkl') 

### Foursquare
Now that we have our location candidates, let's use Foursquare API to get info on restaurants in each neighborhood.

We're interested in venues in 'food' category, but only those that are proper restaurants - coffe shops, pizza places, bakeries etc. are not direct competitors so we don't care about those. So we will include in out list only venues that have 'restaurant' in category name, and we'll make sure to detect and include all the subcategories of specific 'Italian restaurant' category, as we need info on Italian restaurants in the neighborhood.

In [42]:
foursquare_client_id = 'EVPXCM1WKU5151UAWI2143ESYVXP54NJ2JYXNCWW3VZQUBU0' # your Foursquare ID
foursquare_client_secret = 'SOVEIQWXNFKTMKXMZQABIRFGLTJM5SOJ4M455M1AKF01NILU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('My credentails:')
print('CLIENT_ID: ' + foursquare_client_id)
print('CLIENT_SECRET:' + foursquare_client_id)

My credentails:
CLIENT_ID: EVPXCM1WKU5151UAWI2143ESYVXP54NJ2JYXNCWW3VZQUBU0
CLIENT_SECRET:EVPXCM1WKU5151UAWI2143ESYVXP54NJ2JYXNCWW3VZQUBU0


In [43]:
# Category IDs corresponding to Italian restaurants were taken from Foursquare web site (https://developer.foursquare.com/docs/resources/categories):

food_category = '4d4b7105d754a06374d81259' # 'Root' category for all food-related venues

italian_restaurant_categories = ['4bf58dd8d48988d110941735','55a5a1ebe4b013909087cbb6','55a5a1ebe4b013909087cb7c',
                                 '55a5a1ebe4b013909087cba7','55a5a1ebe4b013909087cba1','55a5a1ebe4b013909087cba4',
                                 '55a5a1ebe4b013909087cb95','55a5a1ebe4b013909087cb89','55a5a1ebe4b013909087cb9b',
                                 '55a5a1ebe4b013909087cb98','55a5a1ebe4b013909087cbbf','55a5a1ebe4b013909087cb79',
                                 '55a5a1ebe4b013909087cbb0','55a5a1ebe4b013909087cbb3','55a5a1ebe4b013909087cb74',
                                 '55a5a1ebe4b013909087cbaa','55a5a1ebe4b013909087cb83','55a5a1ebe4b013909087cb8c',
                                 '55a5a1ebe4b013909087cb92','55a5a1ebe4b013909087cb8f','55a5a1ebe4b013909087cb86',
                                 '55a5a1ebe4b013909087cbb9','55a5a1ebe4b013909087cb7f','55a5a1ebe4b013909087cbbc',
                                 '55a5a1ebe4b013909087cb9e','55a5a1ebe4b013909087cbc2','55a5a1ebe4b013909087cbad']

def is_restaurant(categories, specific_filter=None):
    restaurant_words = ['restaurant', 'diner', 'taverna', 'steakhouse']
    restaurant = False
    specific = False
    for c in categories:
        category_name = c[0].lower()
        category_id = c[1]
        for r in restaurant_words:
            if r in category_name:
                restaurant = True
        if 'fast food' in category_name:
            restaurant = False
        if not(specific_filter is None) and (category_id in specific_filter):
            specific = True
            restaurant = True
    return restaurant, specific

def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
    address = address.replace(', Deutschland', '')
    address = address.replace(', Germany', '')
    return address

def get_venues_near_location(lat, lon, category, client_id, client_secret, radius=500, limit=100):
    version = '20180724'
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        client_id, client_secret, version, lat, lon, category, radius, limit)
    try:
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                   item['venue']['name'],
                   get_categories(item['venue']['categories']),
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),
                   format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]        
    except:
        venues = []
    return venues

In [44]:
# Let's now go over our neighborhood locations and get nearby restaurants; we'll also maintain a dictionary of all found restaurants and all found italian restaurants

import pickle

def get_restaurants(lats, lons):
    restaurants = {}
    italian_restaurants = {}
    location_restaurants = []

    print('Obtaining venues around candidate locations:', end='')
    for lat, lon in zip(lats, lons):
        # Using radius=350 to meke sure we have overlaps/full coverage so we don't miss any restaurant (we're using dictionaries to remove any duplicates resulting from area overlaps)
        venues = get_venues_near_location(lat, lon, food_category, foursquare_client_id, foursquare_client_secret, radius=350, limit=100)
        area_restaurants = []
        for venue in venues:
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            is_res, is_italian = is_restaurant(venue_categories, specific_filter=italian_restaurant_categories)
            if is_res:
                x, y = lonlat_to_xy(venue_latlon[1], venue_latlon[0])
                restaurant = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], venue_address, venue_distance, is_italian, x, y)
                if venue_distance<=300:
                    area_restaurants.append(restaurant)
                restaurants[venue_id] = restaurant
                if is_italian:
                    italian_restaurants[venue_id] = restaurant
        location_restaurants.append(area_restaurants)
        print(' .', end='')
    print(' done.')
    return restaurants, italian_restaurants, location_restaurants

# Try to load from local file system in case we did this before
restaurants = {}
italian_restaurants = {}
location_restaurants = []
loaded = False
try:
    with open('restaurants_350.pkl', 'rb') as f:
        restaurants = pickle.load(f)
    with open('italian_restaurants_350.pkl', 'rb') as f:
        italian_restaurants = pickle.load(f)
    with open('location_restaurants_350.pkl', 'rb') as f:
        location_restaurants = pickle.load(f)
    print('Restaurant data loaded.')
    loaded = True
except:
    pass

# If load failed use the Foursquare API to get the data
if not loaded:
    restaurants, italian_restaurants, location_restaurants = get_restaurants(latitudes, longitudes)
    
    # Let's persists this in local file system
    with open('restaurants_350.pkl', 'wb') as f:
        pickle.dump(restaurants, f)
    with open('italian_restaurants_350.pkl', 'wb') as f:
        pickle.dump(italian_restaurants, f)
    with open('location_restaurants_350.pkl', 'wb') as f:
        pickle.dump(location_restaurants, f)
        

Restaurant data loaded.


In [45]:
import numpy as np

print('Total number of restaurants:', len(restaurants))
print('Total number of Italian restaurants:', len(italian_restaurants))
print('Percentage of Italian restaurants: {:.2f}%'.format(len(italian_restaurants) / len(restaurants) * 100))
print('Average number of restaurants in neighborhood:', np.array([len(r) for r in location_restaurants]).mean())

Total number of restaurants: 355
Total number of Italian restaurants: 45
Percentage of Italian restaurants: 12.68%
Average number of restaurants in neighborhood: 0.873626373626


In [46]:
print('List of all restaurants')
print('-----------------------')
for r in list(restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(restaurants))

List of all restaurants
-----------------------
('4f75bd10e4b0b24b0c0fb37d', 'Adana', 43.724966, 7.247757, '29 avenue Saint Sylvestre, 06100 Nice, France', 310, False, -124476.20432767842, 4870618.76408944)
('4c3ee693db3b1b8d67fe6595', 'MiAM', 43.70458024899135, 7.263060013301399, 'France', 219, False, -123455.83835550747, 4868237.895659781)
('4e786e05d22dc865f0ebc426', 'Nikaïa', 43.68839322776762, 7.19813065306297, 'Nice, France', 202, False, -128859.53897042549, 4866932.803206122)
('4b6f2ed6f964a520a4e22ce3', "Noori's", 43.69756158690329, 7.265270654270972, '1 place Grimaldi, 06000 Nice, France', 330, False, -123350.83527888497, 4867441.361563542)
('4d9b6a3e7668f04d939cd193', 'Restaurant Barracuda', 43.695183758510105, 7.260974334687683, '2 Rue Meyerbeer, 06000 Nice, France', 85, False, -123721.97283839132, 4867209.680127582)
('4e89a112cc217acac89cc20a', 'Le Grain De Sel', 43.67304137607981, 7.224940458659901, 'Nice, France', 258, False, -126858.85988791357, 4865022.899370913)
('4d98

In [47]:
print('List of Italian restaurants')
print('---------------------------')
for r in list(italian_restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(italian_restaurants))

List of Italian restaurants
---------------------------
('4bc8b91715a7ef3bdfba7bda', 'La Piazza', 43.7007183740431, 7.287661125037503, '2 Boulevard De Riquier, 06300 Nice, France', 176, True, -121513.0875644302, 4867622.976128765)
('56a2b3d6498e97932102ea50', 'ILLIA PASTA', 43.6981907, 7.2776347, '4 Rue Droite, 06300 Nice, France', 185, True, -122347.60103391926, 4867417.80946469)
('4bbf74fa006dc9b61aa2fc3f', 'Le Luna Rossa', 43.69808559005769, 7.2714946585247935, '3 rue Chauvain, 06000 Nice, France', 348, True, -122843.64933434105, 4867452.5198966935)
('4bb106f0f964a5200b743ce3', 'La Voglia', 43.69557691550856, 7.273364961147308, '2 rue Saint-François de Paule, 06300 Nice, France', 277, True, -122719.00919756072, 4867159.658957819)
('4cc9550a4650a35dbd988d1e', 'La Focaccia', 43.70127777456443, 7.27713989502384, '6 rue Delille, 06000 Nice, France', 192, True, -122355.35142861365, 4867764.53598759)
('4b826a75f964a5201cd230e3', 'Portovenere', 43.696178929604486, 7.265161861210365, '12 ru

In [54]:
print('Restaurants around location')
print('---------------------------')
for i in range(100, 110):
    rs = location_restaurants[i][:8]
    names = ', '.join([r[1] for r in rs])
    print('Restaurants around location {}: {}'.format(i+1, names))

Restaurants around location
---------------------------
Restaurants around location 101: le mesclun
Restaurants around location 102: 
Restaurants around location 103: 
Restaurants around location 104: 
Restaurants around location 105: 
Restaurants around location 106: 
Restaurants around location 107: 
Restaurants around location 108: 
Restaurants around location 109: 
Restaurants around location 110: La Rascasse (AMMAC)


In [55]:
map_nice = folium.Map(location=nice_center, zoom_start=13)
folium.Marker(nice_center, popup='Nice Center').add_to(map_nice)
for res in restaurants.values():
    lat = res[2]; lon = res[3]
    is_italian = res[6]
    color = 'red' if is_italian else 'blue'
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color, fill_opacity=1).add_to(map_nice)
map_nice


Looking good. So now we have all the restaurants in area within few kilometers from Nice Center, and we know which ones are Italian restaurants! We also know which restaurants exactly are in vicinity of every neighborhood candidate center.

This concludes the data gathering phase - we're now ready to use this data for analysis to produce the report on optimal locations for a new Italian restaurant!

## Methodology <a name="methodology"></a>

In this project we will direct our efforts on detecting areas of Nice that have low restaurant density, particularly those with low number of Italian restaurants. We will limit our analysis to area ~3km around city center.

In first step we have collected the required **data: location and type (category) of every restaurant within 3km from Nice center**. We have also **identified Italian restaurants** (according to Foursquare categorization).

Second step in our analysis will be calculation and exploration of '**restaurant density**' across different areas of Nice - we will use **heatmaps** to identify a few promising areas close to center with low number of restaurants in general (*and* no Italian restaurants in vicinity) and focus our attention on those areas.

In third and final step we will focus on most promising areas and within those create **clusters of locations that meet some basic requirements** established in discussion with stakeholders: we will take into consideration locations with **no more than two restaurants in radius of 150 meters**, and we want locations **without Italian restaurants in radius of 400 meters**. We will present map of all such locations but also create clusters (using **k-means clustering**) of those locations to identify general zones / neighborhoods / addresses which should be a starting point for final 'street level' exploration and search for optimal venue location by stakeholders.


## Analysis <a name="analysis"></a>

Let's perform some basic explanatory data analysis and derive some additional info from our raw data. First let's count the **number of restaurants in every area candidate**:

In [56]:
location_restaurants_count = [len(res) for res in location_restaurants]

df_locations['Restaurants in area'] = location_restaurants_count

print('Average number of restaurants in every area with radius=300m:', np.array(location_restaurants_count).mean())

df_locations.head(10)

Average number of restaurants in every area with radius=300m: 0.873626373626


,Address,Distance from center,Latitude,Longitude,X,Y,Restaurants in area,Distance to Italian restaurant
0,"Gambetta, Nice, Alpes-Maritimes, Provence-Alpe...",1997.498436,43.683435,7.263212,-123664.117616,4.865887e+06,0,1309.249219
1,"Massena, Nice, Alpes-Maritimes, Provence-Alpes...",1946.792233,43.683602,7.265670,-123464.117616,4.865887e+06,0,1396.779648
2,"Massena, Nice, Alpes-Maritimes, Provence-Alpes...",1915.724406,43.683770,7.268129,-123264.117616,4.865887e+06,0,1356.369721
3,"Massena, Nice, Alpes-Maritimes, Provence-Alpes...",1905.255888,43.683937,7.270588,-123064.117616,4.865887e+06,0,1289.832067
4,"Massena, Nice, Alpes-Maritimes, Provence-Alpes...",1915.724406,43.684104,7.273047,-122864.117616,4.865887e+06,0,1252.036223
5,"Massena, Nice, Alpes-Maritimes, Provence-Alpes...",1946.792233,43.684272,7.275506,-122664.117616,4.865887e+06,0,1245.601318
6,"Massena, Nice, Alpes-Maritimes, Provence-Alpes...",1997.498436,43.684439,7.277965,-122464.117616,4.865887e+06,0,1231.652592
7,"Grosso, Nice, Alpes-Maritimes, Provence-Alpes-...",1951.922130,43.684729,7.259324,-123964.117616,4.866061e+06,0,1057.624319
8,"Gambetta, Nice, Alpes-Maritimes, Provence-Alpe...",1868.154169,43.684897,7.261782,-123764.117616,4.866061e+06,0,1111.498569
9,"Massena, Nice, Alpes-Maritimes, Provence-Alpes...",1802.775638,43.685064,7.264241,-123564.117616,4.866061e+06,0,1196.782745


OK, now let's calculate the **distance to nearest Italian restaurant from every area candidate center** (not only those within 300m - we want distance to closest one, regardless of how distant it is).

In [57]:
distances_to_italian_restaurant = []

for area_x, area_y in zip(xs, ys):
    min_distance = 10000
    for res in italian_restaurants.values():
        res_x = res[7]
        res_y = res[8]
        d = calc_xy_distance(area_x, area_y, res_x, res_y)
        if d<min_distance:
            min_distance = d
    distances_to_italian_restaurant.append(min_distance)

df_locations['Distance to Italian restaurant'] = distances_to_italian_restaurant

In [58]:
df_locations.head(10)

,Address,Distance from center,Latitude,Longitude,X,Y,Restaurants in area,Distance to Italian restaurant
0,"Gambetta, Nice, Alpes-Maritimes, Provence-Alpe...",1997.498436,43.683435,7.263212,-123664.117616,4.865887e+06,0,1309.249219
1,"Massena, Nice, Alpes-Maritimes, Provence-Alpes...",1946.792233,43.683602,7.265670,-123464.117616,4.865887e+06,0,1396.779648
2,"Massena, Nice, Alpes-Maritimes, Provence-Alpes...",1915.724406,43.683770,7.268129,-123264.117616,4.865887e+06,0,1356.369721
3,"Massena, Nice, Alpes-Maritimes, Provence-Alpes...",1905.255888,43.683937,7.270588,-123064.117616,4.865887e+06,0,1289.832067
4,"Massena, Nice, Alpes-Maritimes, Provence-Alpes...",1915.724406,43.684104,7.273047,-122864.117616,4.865887e+06,0,1252.036223
5,"Massena, Nice, Alpes-Maritimes, Provence-Alpes...",1946.792233,43.684272,7.275506,-122664.117616,4.865887e+06,0,1245.601318
6,"Massena, Nice, Alpes-Maritimes, Provence-Alpes...",1997.498436,43.684439,7.277965,-122464.117616,4.865887e+06,0,1231.652592
7,"Grosso, Nice, Alpes-Maritimes, Provence-Alpes-...",1951.922130,43.684729,7.259324,-123964.117616,4.866061e+06,0,1057.624319
8,"Gambetta, Nice, Alpes-Maritimes, Provence-Alpe...",1868.154169,43.684897,7.261782,-123764.117616,4.866061e+06,0,1111.498569
9,"Massena, Nice, Alpes-Maritimes, Provence-Alpes...",1802.775638,43.685064,7.264241,-123564.117616,4.866061e+06,0,1196.782745


In [60]:
print('Average distance to closest Italian restaurant from each area center:', df_locations['Distance to Italian restaurant'].mean())

Average distance to closest Italian restaurant from each area center: 578.4535245657548


OK, so **on average Italian restaurant can be found within ~580m** from every area center candidate. That's fairly close, so we need to filter our areas carefully!

Let's create a map showing **heatmap / density of restaurants** and try to extract some meaningful info from that. Also, let's show a few circles indicating distance of 0.5 km, 1 km and 1.5 km from Nice Center.

In [61]:
restaurant_latlons = [[res[2], res[3]] for res in restaurants.values()]

italian_latlons = [[res[2], res[3]] for res in italian_restaurants.values()]

In [62]:
from folium import plugins
from folium.plugins import HeatMap

map_nice = folium.Map(location=nice_center, zoom_start=13)
folium.TileLayer('cartodbpositron').add_to(map_nice) #cartodbpositron cartodbdark_matter
HeatMap(restaurant_latlons).add_to(map_nice)
folium.Marker(nice_center).add_to(map_nice)
folium.Circle(nice_center, radius=500, fill=False, color='white').add_to(map_nice)
folium.Circle(nice_center, radius=1000, fill=False, color='white').add_to(map_nice)
folium.Circle(nice_center, radius=1500, fill=False, color='white').add_to(map_nice)

map_nice


Looks like restaurants are frequent in **Promenade des Anglais, Vieux Nice and Carré d'OR**. 

Let's create another heatmap map showing **heatmap/density of Italian restaurants** only.

In [63]:
map_nice = folium.Map(location=nice_center, zoom_start=13)
folium.TileLayer('cartodbpositron').add_to(map_nice) #cartodbpositron cartodbdark_matter
HeatMap(italian_latlons).add_to(map_nice)
folium.Marker(nice_center).add_to(map_nice)
folium.Circle(nice_center, radius=1000, fill=False, color='white').add_to(map_nice)
folium.Circle(nice_center, radius=2000, fill=False, color='white').add_to(map_nice)
folium.Circle(nice_center, radius=3000, fill=False, color='white').add_to(map_nice)
map_nice


This map is not so 'hot' (Italian restaurants represent a subset of ~12% of all restaurants in Nice). **There is almost no Italian restaurants in the center, the north and in the west.**

In [64]:
good_res_count = np.array((df_locations['Restaurants in area']<=2))
print('Locations with no more than two restaurant nearby:', good_res_count.sum())

good_ita_distance = np.array(df_locations['Distance to Italian restaurant']>=150)
print('Locations with no Italian restaurants within 150m:', good_ita_distance.sum())

good_locations = np.logical_and(good_res_count, good_ita_distance)
print('Locations with both conditions met:', good_locations.sum())

df_good_locations = df_locations[good_locations]


Locations with no more than two restaurant nearby: 341
Locations with no Italian restaurants within 150m: 327
Locations with both conditions met: 306


Let's see how this looks on a map.

In [65]:
good_latitudes = df_good_locations['Latitude'].values
good_longitudes = df_good_locations['Longitude'].values

good_locations = [[lat, lon] for lat, lon in zip(good_latitudes, good_longitudes)]

map_nice = folium.Map(location=nice_center, zoom_start=13)
folium.TileLayer('cartodbpositron').add_to(map_nice)
HeatMap(restaurant_latlons).add_to(map_nice)
folium.Circle(nice_center, radius=2500, color='white', fill=True, fill_opacity=0.6).add_to(map_nice)
folium.Marker(nice_center).add_to(map_nice)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_nice) 
map_nice


Looking good. We now have a bunch of locations fairly close to the city center, and we know that each of those locations has no more than two restaurants in radius of 400m, and no Italian restaurant closer than 150m. Any of those locations is a potential candidate for a new Italian restaurant, at least based on nearby competition.

Let's now show those good locations in a form of heatmap:

In [66]:
map_nice = folium.Map(location=nice_center, zoom_start=14)
HeatMap(good_locations, radius=25).add_to(map_nice)
folium.Marker(nice_center).add_to(map_nice)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_nice)
map_nice


Looking good. What we have now is a clear indication of zones with low number of restaurants in vicinity, and *no* Italian restaurants at all nearby.

Let us now **cluster** those locations to create **centers of zones containing good locations**. Those zones, their centers and addresses will be the final result of our analysis. 

In [67]:
from sklearn.cluster import KMeans

number_of_clusters = 15

good_xys = df_good_locations[['X', 'Y']].values
kmeans = KMeans(n_clusters=number_of_clusters, random_state=0).fit(good_xys)

cluster_centers = [xy_to_lonlat(cc[0], cc[1]) for cc in kmeans.cluster_centers_]

map_nice = folium.Map(location=nice_center, zoom_start=14)
folium.TileLayer('cartodbpositron').add_to(map_nice)
HeatMap(restaurant_latlons).add_to(map_nice)
folium.Circle(nice_center, radius=2500, color='white', fill=True, fill_opacity=0.4).add_to(map_nice)
folium.Marker(nice_center).add_to(map_nice)
for lon, lat in cluster_centers:
    folium.Circle([lat, lon], radius=500, color='green', fill=True, fill_opacity=0.25).add_to(map_nice) 
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_nice)
map_nice


Not bad - our clusters represent groupings of most of the candidate locations and cluster centers are placed nicely in the middle of the zones 'rich' with location candidates.

Addresses of those cluster centers will be a good starting point for exploring the neighborhoods to find the best possible location based on neighborhood specifics.

Let's see those zones on a city map without heatmap, using shaded areas to indicate our clusters:

In [69]:
map_nice = folium.Map(location=nice_center, zoom_start=14)
folium.Marker(nice_center).add_to(map_nice)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.Circle([lat, lon], radius=250, color='#00000000', fill=True, fill_color='#0066ff', fill_opacity=0.07).add_to(map_nice)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_nice)
for lon, lat in cluster_centers:
    folium.Circle([lat, lon], radius=500, color='green', fill=False).add_to(map_nice) 

map_nice


Finaly, let's reverse geocode those candidate area centers to get the addresses which can be presented to stakeholders.

In [72]:
candidate_area_addresses = []
print('==============================================================')
print('Addresses of centers of areas recommended for further analysis')
print('==============================================================\n')
for lon, lat in cluster_centers:
    addr = get_address(lat, lon).replace(', Germany', '')
    candidate_area_addresses.append(addr)    
    x, y = lonlat_to_xy(lon, lat)
    d = calc_xy_distance(x, y, nice_center_x, nice_center_y)
    print('{}{} => {:.1f}km from Nice Center'.format(addr, ' '*(50-len(addr)), d/1000))

Addresses of centers of areas recommended for further analysis

Villa Saint-George, Avenue Mirabeau, Libération, Nice, Alpes-Maritimes, Provence-Alpes-Côte d'Azur, France métropolitaine, 06000, France => 0.9km from Nice Center
Raubà capeu, Promenade des Anglais, Vieux Nice, Nice, Alpes-Maritimes, Provence-Alpes-Côte d'Azur, France métropolitaine, 06300, France => 1.6km from Nice Center
Massena, Nice, Alpes-Maritimes, Provence-Alpes-Côte d'Azur, France métropolitaine, 06046, France => 1.6km from Nice Center
Avenue des Diables Bleus, Riquier, Nice, Alpes-Maritimes, Provence-Alpes-Côte d'Azur, France métropolitaine, 06300, France => 1.6km from Nice Center
Escalier, Avenue Dauphiné, L'Archet, La Madeleine, Nice, Alpes-Maritimes, Provence-Alpes-Côte d'Azur, France métropolitaine, 06100, France => 1.7km from Nice Center
L'Archet, Grosso, Nice, Alpes-Maritimes, Provence-Alpes-Côte d'Azur, France métropolitaine, 06046, France => 1.6km from Nice Center
15, Boulevard de Cessole, Saint-Barthélémy

This concludes our analysis. We have created 15 addresses representing centers of zones containing locations with low number of restaurants and no Italian restaurants nearby, all zones being fairly close to city center. 

In [75]:
map_nice = folium.Map(location=nice_center, zoom_start=14)
folium.Circle(nice_center, radius=50, color='red', fill=True, fill_color='red', fill_opacity=1).add_to(map_nice)
for lonlat, addr in zip(cluster_centers, candidate_area_addresses):
    folium.Marker([lonlat[1], lonlat[0]], popup=addr).add_to(map_nice) 
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.Circle([lat, lon], radius=250, color='#0000ff00', fill=True, fill_color='#0066ff', fill_opacity=0.05).add_to(map_nice)
map_nice


## Results and Discussion <a name="results"></a>

Our analysis shows that are not many restaurants in Nice and even fewer Italian restaurants.
Highest concentration of restaurants was detected in Promenade des Anglais, Vieux Nice and Carré d'OR. These neighborhoods may be more attractive for tourists.

We first created a dense grid of location candidates (spaced 100m appart); those locations were then filtered so that those with more than two restaurants in radius of 150m and those with an Italian restaurant closer than 400m were removed.
Those location candidates were then clustered to create zones of interest which contain greatest number of location candidates. Addresses of centers of those zones were also generated using reverse geocoding to be used as markers/starting points for more detailed local analysis based on other factors.

Result of all this is 15 zones containing largest number of potential new restaurant locations based on number of and distance to existing venues - both restaurants in general and Italian restaurants particularly. This, of course, does not imply that those zones are actually optimal locations for a new restaurant! Purpose of this analysis was to only provide info on areas close to Nice center but not crowded with existing restaurants (particularly Italian) - it is entirely possible that there is a very good reason for small number of restaurants in any of those areas, reasons which would make them unsuitable for a new restaurant regardless of lack of competition in the area. Recommended zones should therefore be considered only as a starting point for more detailed analysis which could eventually result in location which has not only no nearby competition but also other factors taken into account and all other relevant conditions met.


## Conclusion <a name="conclusion"></a>

Purpose of this project was to identify Nice areas close to center with low number of restaurants (particularly Italian restaurants) in order to aid stakeholders in narrowing down the search for optimal location for a new Italian restaurant. By calculating restaurant density distribution from Foursquare data we have generated extensive collection of locations which satisfy some basic requirements regarding existing nearby restaurants. Clustering of those locations was then performed in order to create major zones of interest (containing greatest number of potential locations) and addresses of those zone centers were created to be used as starting points for final exploration by stakeholders.

Final decision on optimal restaurant location will be made by stakeholders based on specific characteristics of neighborhoods and locations in every recommended zone, taking into consideration additional factors like attractiveness of each location (proximity to park or water), levels of noise / proximity to major roads, real estate availability, prices, social and economic dynamics of every neighborhood etc.
